### S3로 데이터 적재

In [1]:
from pydatafabric.vault_utils import get_secrets

aws_info = get_secrets(mount_point="datafabric",path="mlops/access/data_loader")

In [2]:
from pydatafabric.ye import get_spark
import os

#valut를 통해 aws credential 정보 로딩
# os.environ["AWS_ACCESS_KEY_ID"] = aws_info["aws_access_key_id"]
# os.environ["AWS_SECRET_ACCESS_KEY"] = aws_info["aws_secret_access_key"]

# !printenv | grep -i aws

spark = get_spark()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/21 21:36:59 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
22/04/21 21:36:59 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
22/04/21 21:36:59 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
22/04/21 21:36:59 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [3]:
spark.conf.set("fs.s3a.fast.upload.buffer", "disk")
spark.conf.set("fs.s3a.buffer.dir", "/tmp")
# spark.conf.set("fs.s3a.fast.upload", "true")
# spark.conf.set("fs.s3a.fast.upload.active.blocks", 1)
spark.conf.set("fs.s3a.access.key", aws_info["aws_access_key_id"])
spark.conf.set("fs.s3a.secret.key", aws_info["aws_secret_access_key"])

In [4]:
review_keywords = spark.read.format('bigquery') \
  .option('table', 'smart-ruler-304409:product_attribute_module.app_review_keywords') \
  .load()
review_keywords.createOrReplaceTempView('review_keywords')

In [5]:
query="select * from review_keywords"

In [6]:
df = spark.sql(query)
df.show(5)

+-------------+--------+----------+------------------------------------+------------------------+----------------------------------+----------+-------------------------------+
|      prdt_cd|store_cd|order_date|                            sku_name|           link_sku_name|                           comment|score_text|                       keywords|
+-------------+--------+----------+------------------------------------+------------------------+----------------------------------+----------+-------------------------------+
|1102150000000|    1018|  20220102|(후레쉬팩)미국냉장초이스척아이로스트|(후레쉬팩)미국산CH갈비살|    기름기가 많아서 
먹기 불편했음|       Bad|    기름기 많아서 먹기 불편했음|
|1102150000000|    1108|  20220101|(후레쉬팩)미국냉장초이스척아이로스트|(후레쉬팩)미국산CH갈비살| 기름이 너무너무 많아요.버린 기...|       Bad|       기름 많아요 기름 무게 만|
|1102150000000|    1108|  20220101|(후레쉬팩)미국냉장초이스척아이로스트|(후레쉬팩)미국산CH갈비살|세일전주에 샀을때 손질해보니 기...|       Bad|전주 때 손질 기름 근육 고기 ...|
|1102150000000|    1048|  20220101|(후레쉬팩)미국냉장초이스척아이로스트|(후레쉬팩)미국산CH갈비살| 고기가 상태가 별로였어요
빨리 ...|      

In [7]:
env = ["dev"]

In [8]:
for k in env:
    s3_path = f"s3a://mlops-loader-dev/review_keywords"
    print(s3_path)

    spark.sql(query).write.mode("overwrite").parquet(s3_path)
    # review_keywords.write.mode("overwrite").parquet(s3_path)


s3a://mlops-loader-dev/review_keywords


In [9]:
spark.stop()